# SEIRX model runs for test technology and screening frequency investigation

In [67]:
import networkx as nx
import pandas as pd
from os.path import join
import json

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

# custom functionality
import os
from os.path import expanduser
home = expanduser("~")
os.chdir(home+'/agent_based_COVID_SEIRX/src/')
os.getcwd()
from scseirx.model_nursing_home import SEIRX_nursing_home # agent-based simulation
from scseirx import analysis_functions as af # custom analysis functions
import data_creation_functions as dcf

from importlib import reload

## Parameters

### Simulation parameters and measures in place

In [68]:

os.getcwd()

'/Users/ledebur/agent_based_COVID_SEIRX/src'

In [69]:
with open(home+'/nursing_home_SEIRX/params/screening_frequency_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open(home+'/nursing_home_SEIRX/params/screening_frequency_measures.json') as json_file:
    measures = json.load(json_file)


In [70]:
measures

{'testing': 0,
 'preventive_screening_test_type': 'same_day_antigen',
 'diagnostic_test_type': 'one_day_PCR',
 'K1_contact_types': ['close', 'intermediate'],
 'quarantine_duration': 10,
 'employee_mask': False,
 'resident_mask': False,
 'employee_vaccination': 0.5,
 'resident_vaccination': 0.8,
 'follow_up_testing_interval': None,
 'liberating_testing': False,
 'transmission_risk_ventilation_modifier': 1,
 'transmission_risk_vaccination_modifier': {'reception': 0.6,
  'transmission': 0.3},
 'mask_filter_efficiency': {'exhale': 0.5, 'inhale': 0.7}}

### Screening parameters

In [79]:
# test technologies (and test result turnover times) used in the
# different scenarios
test_types = ['same_day_antigen']#, 'same_day_LAMP', 'same_day_PCR',\
              #'one_day_PCR', 'two_day_PCR']

e_vaccination_probabilities = [ 0.5, 0.6, 0.7, 0.8]
r_vaccination_probabilities = [ 0.5, 0.6, 0.7, 0.8]

# specifies, whether the index case will be introduced via an
# employee or a resident
index_cases = ['employee', 'resident']

# resident and employee streening intervals (in days)
#e_screen_range = [2, 3, 7, None]
#r_screen_range = [2, 3, 7, None]

screening_params = [(i, j, k, l)\
              for i in test_types \
              for j in index_cases \
              for k in e_vaccination_probabilities \
              for l in r_vaccination_probabilities]

print('there are {} parameter combinations'.format(len(screening_params)))

there are 32 parameter combinations


In [80]:
def run(param_list, N_runs=100):
    '''
    Runs an ensemble of simulations and collects observable statistics. To be 
    run in parallel on many workers. Note: I/O paths and the number of runs per 
    ensemble hare hard coded here, because I only want to pass the parameter 
    values that are being screened in the simulation run to the function via the
    parallel processing interface.
    
    Parameters:
    -----------
    param_list : iterable
        Iterable that contains the values for the parameters test_type, 
        index_case, e_screen_range and r_screen_range that are passed to the
        simulation.
    N_runs : integer
        Number of runs per ensemble.
        
    Returns:
    --------
    row : dictionary
        Dictionary of the ensemble statistics of the observables.
    '''

    # paths for data I/O
    contact_network_src = home+'/nursing_home_SEIRX/data/contact_networks'

    # extract the simulation parameters from the parameter list
    test_type, index_case, e_vaccination_probability, r_vaccination_probability = param_list

    # run the ensemble with the given simulation parameters 
    ensemble_results = dcf.run_ensemble(N_runs, test_type, index_case, e_screen_interval, r_screen_interval,
                e_vaccination_probability, r_vaccination_probability, measures,
                simulation_params, contact_network_src)

    # calculate the ensemble statistics
    row = dcf.evaluate_ensemble(ensemble_results, test_type, index_case, e_screen_interval, r_screen_interval,
                                e_vaccination_probability, r_vaccination_probability)

    return row

In [81]:
contact_network_src = home+'/nursing_home_SEIRX/data/contact_networks'

In [82]:
dcf.run_model('same_day_antigen', 'employee', None, None, 0.3, 0.6, measures, simulation_params, contact_network_src)

In [83]:
e_screen_interval = None
r_screen_interval = None

In [84]:
%%time
number_of_cores = psutil.cpu_count(logical=True) - 2 
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
dst = home+'/nursing_home_SEIRX/data/simulation_results'
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['test_type', 'index_case',
              'resident_vaccination_probability', 'employee_vaccination_probability']
              #'resident_screen_interval', 'employee_screen_interval']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

#results.to_csv(join(dst,'simulations_test_technology_screening_interval.csv'),
               #index=False)
results.head(3)

100%|██████████| 32/32 [00:45<00:00,  1.41s/it]


CPU times: user 393 ms, sys: 83.4 ms, total: 476 ms
Wall time: 45.6 s


,test_type,index_case,resident_vaccination_probability,employee_vaccination_probability,N_diagnostic_tests_0.025,N_diagnostic_tests_0.10,N_diagnostic_tests_0.25,N_diagnostic_tests_0.75,N_diagnostic_tests_0.90,N_diagnostic_tests_0.975,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,same_day_antigen,employee,0.8,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,12.649666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,same_day_antigen,employee,0.7,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,14.576427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,same_day_antigen,employee,0.6,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,15.542890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
dst = home+'/nursing_home_SEIRX/data/simulation_results'

In [86]:
results.to_csv(join(dst,'simulations_test_technology_screening_interval.csv'),
               index=False)